In [1]:
#!pip uninstall -y torch

In [ ]:
!pip install -q torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0 --extra-index-url https://download.pytorch.org/whl/cu126


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 129.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 213.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 117.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 53.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 74.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 98.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 128.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 105.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/216.6 MB ? eta -:--:--

In [ ]:
!pip install -q -U transformers bitsandbytes
!pip install -q -U langchain langchain-community langchain-huggingface chromadb sentence-transformers
!pip install -q -U accelerate

In [ ]:
import json
import torch
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
"""import json
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma"""

# 1. Load dữ liệu
try:
    with open('/kaggle/input/medicine-data/merged.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
except FileNotFoundError:
    # Dữ liệu mẫu minh họa
    data = [
        {"category": "Thuốc", "title": "Panadol", "content": "Panadol chứa hoạt chất Paracetamol. Thuốc có tác dụng hạ sốt, giảm đau nhẹ...", "source": "Vinmec"},
        {"category": "Bệnh", "title": "Viêm phổi", "content": "Viêm phổi là tình trạng nhiễm trùng phổi. Triệu chứng gồm ho, sốt cao...", "source": "BoYTe"}
    ]
# Thống kê dữ liệu đầu vào
print(f"\n{'='*60}")
print(f"📊 THỐNG KÊ DỮ LIỆU ĐẦU VÀO")
print(f"{'='*60}")
print(f"📝 Tổng số bản ghi: {len(data)}")
# 2. Chuyển đổi JSON thành LangChain Documents
documents = []
for item in data:
    page_content = item['content']
    
    metadata = {
        "source": item.get("source", ""),
        "title": item.get("title", ""),
        "category": item.get("category", "") 
    }
    
    documents.append(Document(page_content=page_content, metadata=metadata))

# ==================== BƯỚC 3: CHUNKING (CẢI TIẾN) ====================
print("✂️  Đang chia nhỏ văn bản...")
# CẢI TIẾN 2: Điều chỉnh chunk_size phù hợp với văn bản y tế
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # Giảm xuống 400 cho văn bản y tế (thường ngắn gọn)
    chunk_overlap=80,  # Giảm overlap tương ứng
    separators=["\n\n", "\n", ". ", ", ", " "],
    length_function=len
)
splits = text_splitter.split_documents(documents)
print(f"✅ Đã tạo {len(splits)} chunks")

# ==================== BƯỚC 4: TẠO VECTOR DB ====================
print("\n🔢 Đang tạo embeddings và vector database...")
embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
)

vector_db = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="medical_rag_v3"
)
print("✅ Vector DB đã sẵn sàng!")

In [ ]:
"""import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline"""

# Tên mô hình trên HuggingFace
model_id = "Qwen/Qwen2.5-0.5B-Instruct"

# 1. Tải Tokenizer và Model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto", 
    low_cpu_mem_usage=True
)

# 2. Tạo Pipeline sinh văn bản
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,       
    do_sample=False,           
    repetition_penalty=1.1,   
    return_full_text=False   
)

# 3. Bọc vào LangChain LLM
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

print("✅ Đã tải xong mô hình Qwen2.5-0.5B!")

In [ ]:
# ==================== BƯỚC 6: RERANKER (MỚI) ====================
print("\n🎯 Đang tải Reranker model...")

class CrossEncoderReranker:
    """
    Reranker sử dụng Cross-Encoder để xếp hạng lại documents
    """
    def __init__(self, model_name="cross-encoder/ms-marco-MiniLM-L-6-v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)
        self.model.eval()
    
    def build_rerank_text(self, doc):
        title = doc.metadata.get("title", "")
        content = doc.page_content

        return f"{title}. {content}"

    def rerank(self, query, documents, top_k=3):
        """
        Xếp hạng lại documents dựa trên độ liên quan với query
        
        Args:
            query: Câu hỏi
            documents: List các Document từ retriever
            top_k: Số lượng documents giữ lại
        
        Returns:
            List documents đã được xếp hạng lại
        """
        if not documents:
            return []
        
        # Tạo pairs (query, doc)
        pairs = [[query, self.build_rerank_text(doc)] for doc in documents]
        
        # Tính điểm cho từng pair
        with torch.no_grad():
            inputs = self.tokenizer(
                pairs, 
                padding=True, 
                truncation=True, 
                return_tensors='pt',
                max_length=512
            ).to(self.device)
            
            scores = self.model(**inputs).logits.squeeze(-1).cpu().numpy()
        doc_score_pairs = list(zip(documents, scores))
        doc_score_pairs.sort(key=lambda x: x[1], reverse=True)
        
        # Lấy top_k documents
        reranked_docs = [doc for doc, score in doc_score_pairs[:top_k]]
        
        # In thông tin debug
        print(f"\n🔍 Reranking results:")
        for i, (doc, score) in enumerate(doc_score_pairs[:top_k], 1):
            print(f"  {i}. Score: {score:.4f} | Title: {doc.metadata.get('title', 'N/A')[:50]}")
        
        return reranked_docs

# Khởi tạo reranker
reranker = CrossEncoderReranker()
print("✅ Reranker đã sẵn sàng!")

In [ ]:
# ==================== BƯỚC 7: TẠO RETRIEVER VỚI RERANKER ====================
print("\n🔧 Đang thiết lập retriever...")

# CẢI TIẾN 4: Tăng số documents ban đầu, sau đó rerank
base_retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}  # Lấy 10 docs trước, sau đó rerank xuống 3
)

def retrieve_and_rerank(query):
    """
    Hàm kết hợp retrieval và reranking
    """
    # Bước 1: Retrieval (lấy 10 docs)
    docs = base_retriever.invoke(query)
    
    # Bước 2: Rerank (giữ lại 3 docs tốt nhất)
    reranked_docs = reranker.rerank(query, docs, top_k=3)
    
    return reranked_docs

In [ ]:

# ==================== BƯỚC 8: PROMPT ENGINEERING (CẢI TIẾN) ====================
# CẢI TIẾN 5: Prompt rõ ràng, ngắn gọn hơn
prompt_template = """<|im_start|>system
Bạn là trợ lý y tế AI chuyên nghiệp. Trả lời câu hỏi Đúng/Sai dựa trên văn bản y tế được cung cấp.

QUY TẮC:
- Chỉ trả lời "Đúng" hoặc "Sai"
- Không giải thích thêm
- Nếu văn bản xác nhận thông tin → "Đúng"
- Nếu văn bản phủ nhận hoặc không đề cập → "Sai"
<|im_end|>

<|im_start|>user
Văn bản tham khảo:
{context}

Câu hỏi: {question}

Trả lời (chỉ ghi "Đúng" hoặc "Sai"):
<|im_end|>

<|im_start|>assistant
"""

PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

In [ ]:
# ==================== BƯỚC 9: TẠO RAG CHAIN ====================
def format_docs(docs):
    """Format documents với thông tin metadata"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        formatted.append(f"[Tài liệu {i}]\n{doc.page_content}")
    return "\n\n---\n\n".join(formatted)

# RAG chain với reranker
rag_chain_with_reranker = (
    {
        "context": lambda x: format_docs(retrieve_and_rerank(x)),
        "question": RunnablePassthrough()
    }
    | PROMPT
    | llm
    | StrOutputParser()
)

print("✅ Hệ thống RAG với Reranker đã sẵn sàng!")

In [ ]:
# ==================== BƯỚC 10: HÀM TIỆN ÍCH ====================
def get_final_verdict(response_text):
    """
    Trích xuất câu trả lời Đúng/Sai
    """
    response_text = response_text.strip().lower()
    
    # Ưu tiên từ đầu tiên
    first_word = response_text.split()[0] if response_text else ""
    
    if "đúng" in first_word:
        return "TRUE"
    elif "sai" in first_word:
        return "FALSE"
    
    # Fallback
    if "đúng" in response_text and "sai" not in response_text:
        return "TRUE"
    elif "sai" in response_text and "đúng" not in response_text:
        return "FALSE"
    else:
        return "FALSE"

def answer_question(question_text, verbose=True):
    """
    Trả lời câu hỏi với logging chi tiết
    """
    print(f"\n{'='*80}")
    print(f"❓ Câu hỏi: {question_text}")
    print(f"{'='*80}")
    
    # Generate answer
    response_text = rag_chain_with_reranker.invoke(question_text)
    
    if verbose:
        print(f"\n🤖 AI Raw Output:\n{response_text}")
    
    # Parse verdict
    verdict = get_final_verdict(response_text)
    print(f"\n✅ Kết luận: {verdict}")
    
    return verdict

In [ ]:
# ==================== BƯỚC 11: TEST TRÊN TẬP KIỂM THỬ ====================
def load_test_data(file_path):
    """
    Load test data từ file Excel
    """
    import pandas as pd
    
    try:
        df = pd.read_excel(file_path)
        print(f"✅ Đã load {len(df)} câu hỏi test từ file")
        return df
    except FileNotFoundError:
        print(f"❌ Không tìm thấy file: {file_path}")
        return None
    except Exception as e:
        print(f"❌ Lỗi khi đọc file: {e}")
        return None

def evaluate_on_test_set(test_df, verbose=False, save_results=True):
    """
    Đánh giá hệ thống trên tập test và tính accuracy
    
    Args:
        test_df: DataFrame chứa test data
        verbose: In chi tiết từng câu hỏi
        save_results: Lưu kết quả ra file
    
    Returns:
        accuracy: Độ chính xác (%)
    """
    if test_df is None:
        return 0.0
    
    results = []
    correct = 0
    total = 0
    
    print("\n" + "="*80)
    print("🧪 BẮT ĐẦU ĐÁNH GIÁ TRÊN TẬP TEST")
    print("="*80)
    print(f"Tổng số câu hỏi: {len(test_df)}\n")
    
    for idx, row in test_df.iterrows():
        question = row['Mệnh đề Câu hỏi (VIETNAMESE TEXT ONLY)']
        ground_truth = str(row['Đáp án (TRUE/FALSE)']).upper()
        
        # Kiểm tra ground truth hợp lệ
        if ground_truth not in ['TRUE', 'FALSE']:
            print(f"⚠️  Bỏ qua câu {idx+1}: Ground truth không hợp lệ ({ground_truth})")
            continue
        
        total += 1
        
        # Dự đoán
        if verbose:
            print(f"\n{'='*80}")
            print(f"Câu {idx+1}/{len(test_df)}")
            print(f"{'='*80}")
            print(f"❓ Câu hỏi: {question}")
            print(f"✅ Đáp án đúng: {ground_truth}")
        
        try:
            # Gọi hệ thống RAG
            response_text = rag_chain_with_reranker.invoke(question)
            prediction = get_final_verdict(response_text)
            
            # Kiểm tra đúng/sai
            is_correct = (prediction == ground_truth)
            if is_correct:
                correct += 1
            
            if verbose:
                print(f"🤖 Dự đoán: {prediction}")
                print(f"{'✅ ĐÚNG' if is_correct else '❌ SAI'}")
                if not is_correct:
                    print(f"   Raw output: {response_text[:200]}...")
            else:
                # Hiển thị progress
                status = "✅" if is_correct else "❌"
                print(f"{status} Câu {total}: {prediction} (GT: {ground_truth})")
            
            # Lưu kết quả
            results.append({
                'STT': idx + 1,
                'Question': question,
                'Ground_Truth': ground_truth,
                'Prediction': prediction,
                'Correct': is_correct,
                'Raw_Output': response_text
            })
            
        except Exception as e:
            print(f"❌ Lỗi khi xử lý câu {idx+1}: {e}")
            results.append({
                'STT': idx + 1,
                'Question': question,
                'Ground_Truth': ground_truth,
                'Prediction': 'ERROR',
                'Correct': False,
                'Raw_Output': str(e)
            })
    
    # Tính accuracy
    accuracy = (correct / total * 100) if total > 0 else 0
    
    # In kết quả tổng hợp
    print("\n" + "="*80)
    print("📊 KẾT QUẢ ĐÁNH GIÁ")
    print("="*80)
    print(f"Tổng số câu hỏi: {total}")
    print(f"Số câu đúng: {correct}")
    print(f"Số câu sai: {total - correct}")
    print(f"Accuracy: {accuracy:.2f}%")
    
    # Phân tích lỗi theo loại
    if results:
        import pandas as pd
        results_df = pd.DataFrame(results)
        
        print("\n📈 Phân tích chi tiết:")
        
        # Đếm số lượng TRUE/FALSE predictions
        pred_counts = results_df['Prediction'].value_counts()
        print(f"\nPhân bố dự đoán:")
        for pred, count in pred_counts.items():
            print(f"  - {pred}: {count} câu")
        
        # Confusion Matrix
        print(f"\n🎯 Confusion Matrix:")
        confusion = pd.crosstab(
            results_df['Ground_Truth'], 
            results_df['Prediction'],
            rownames=['Ground Truth'],
            colnames=['Prediction'],
            margins=True
        )
        print(confusion)
    
    # Lưu kết quả ra file
    if save_results and results:
        import pandas as pd
        results_df = pd.DataFrame(results)
        output_file = 'test_results.xlsx'
        results_df.to_excel(output_file, index=False)
        print(f"\n✅ Đã lưu kết quả chi tiết vào: {output_file}")
        
        # Lưu báo cáo text
        report_file = 'test_report.txt'
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write("BÁO CÁO ĐÁNH GIÁ HỆ THỐNG RAG\n")
            f.write("="*80 + "\n\n")
            f.write(f"Tổng số câu hỏi: {total}\n")
            f.write(f"Số câu đúng: {correct}\n")
            f.write(f"Số câu sai: {total - correct}\n")
            f.write(f"Accuracy: {accuracy:.2f}%\n\n")
            
            f.write("="*80 + "\n")
            f.write("CÁC CÂU TRẢ LỜI SAI\n")
            f.write("="*80 + "\n\n")
            
            wrong_answers = results_df[results_df['Correct'] == False]
            for idx, row in wrong_answers.iterrows():
                f.write(f"STT: {row['STT']}\n")
                f.write(f"Câu hỏi: {row['Question']}\n")
                f.write(f"Ground Truth: {row['Ground_Truth']}\n")
                f.write(f"Prediction: {row['Prediction']}\n")
                f.write(f"Raw Output: {row['Raw_Output'][:200]}...\n")
                f.write("-"*80 + "\n\n")
        
        print(f"✅ Đã lưu báo cáo vào: {report_file}")
    
    return accuracy

if __name__ == "__main__":
    # Đường dẫn file test
    test_file_path = r"/kaggle/input/datatest/Test_sample.v1.0.xlsx"
    
    # Load test data
    test_df = load_test_data(test_file_path)
    
    if test_df is not None:
        # Chạy evaluation
        accuracy = evaluate_on_test_set(
            test_df, 
            verbose=False,  # Đặt True để xem chi tiết từng câu
            save_results=True
        )
        
        print(f"\n{'='*80}")
        print(f"🎯 ACCURACY CUỐI CÙNG: {accuracy:.2f}%")
        print(f"{'='*80}")
    else:
        print("\n❌ Không thể thực hiện evaluation do không load được test data")


In [ ]:
# ==================== BƯỚC 11: TEST ====================
if __name__ == "__main__":
    # Test với câu hỏi mẫu
    test_questions = [
        "Thoái hóa khớp cùng chậu có thể gây đau lưng, hông, cảm giác tê bì chân.",
        "Tình trạng lồi mắt ở người bệnh Basedow thường sẽ duy trì lâu dài ngay cả khi người bệnh đã được điều trị bình giáp.",
        "Trong cơn sốt rét, các triệu chứng điển hình luôn là rét run, sốt nóng và ra mồ hôi, không có triệu chứng sốt không thành cơn.",
        "Các thuốc điều trị béo phì thường không mang lại hiệu quả cao nếu không kết hợp với chế độ ăn giảm cân và tăng cường vận động thể lực.",
        "Tăng huyết áp hậu sản chỉ là vô căn (không rõ nguyên nhân) và không gây biến chứng nguy hiểm.",
        "Viêm thận bể thận không được coi là NTTNDT phức tạp."
    ]
    
    print("\n" + "="*80)
    print("🧪 BẮT ĐẦU TEST HỆ THỐNG")
    print("="*80)
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n--- Test {i}/{len(test_questions)} ---")
        answer_question(question, verbose=True)